In [1]:
import pinecone
import os

PINECONE_API_KEY = "bf349a22-3a4e-487a-a05a-14f9193ed5e6"
PINECONE_ENV_KEY = "asia-southeast1-gcp-free"

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV_KEY
)

c:\Users\86183\anaconda3\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
index_name = "pinecone-namespacing"

if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)

pinecone.create_index(
    name=index_name,
    dimension=2,
    metric="euclidean",
    shards=1
)
pinecone.list_indexes()

['pinecone-namespacing']

In [3]:
index = pinecone.Index(
    index_name=index_name
)

In [4]:
import pandas as pd

df = pd.DataFrame()
df["id"] = ["Wall-E", "Up", "Ratatouille", "Toy Story"]
df["vector"] = [[1., 1.], [2., 2.], [3., 3.], [4., 4.]]
df

,id,vector
0,Wall-E,"[1.0, 1.0]"
1,Up,"[2.0, 2.0]"
2,Ratatouille,"[3.0, 3.0]"
3,Toy Story,"[4.0, 4.0]"


In [5]:
index.upsert(
    vectors=zip(df["id"], df["vector"])
)

index.describe_index_stats()

{'dimension': 2,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 4}},
 'total_vector_count': 4}

In [7]:
romantic_comedies = ["Wall-E", "Ratatouille"]
romcom_df = df[df["id"].isin(romantic_comedies)]
romcom_df

,id,vector
0,Wall-E,"[1.0, 1.0]"
2,Ratatouille,"[3.0, 3.0]"


In [8]:
index.upsert(
    vectors=zip(romcom_df["id"], romcom_df["vector"]),
    namespace="romantic_comedy"
)

index.describe_index_stats()

{'dimension': 2,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 4},
                'romantic_comedy': {'vector_count': 2}},
 'total_vector_count': 6}

In [9]:
query_results = index.query(
    queries=df[df["id"] == "Wall-E"]["vector"],
    top_k=2
)
query_results

{'results': [{'matches': [{'id': 'Wall-E', 'score': 0.0, 'values': []},
                          {'id': 'Up', 'score': 1.99999905, 'values': []}],
              'namespace': ''}]}

In [10]:
query_results = index.query(
    queries=df[df["id"] == "Wall-E"]["vector"],
    top_k=2,
    namespace="romantic_comedy"
)
query_results

{'results': [{'matches': [{'id': 'Wall-E', 'score': 0.0, 'values': []},
                          {'id': 'Ratatouille',
                           'score': 7.99999809,
                           'values': []}],
              'namespace': 'romantic_comedy'}]}

In [11]:
pinecone.delete_index(index_name)